In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import keras
import keras.backend as K
sys.path.append("data")
from data.brainds import BrainTumorDataset
from models.unet import *
from callbacks import ImageHistory

Using TensorFlow backend.


In [3]:
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import keras

In [4]:
data_directory = './data/datasets/brain_tumor'
weight_file = './weights.hdf5'
tensor_board_root_directory = './brain_tumor_logs'

In [5]:
dataset = BrainTumorDataset(data_directory)

In [6]:
#dataset.download()

In [7]:
#dataset.prepare()

In [8]:
training_dataset_size = dataset.get_dataset_size('training')
validation_dataset_size = dataset.get_dataset_size('validation')
test_dataset_size = dataset.get_dataset_size('test')
learning_rate = 1e-4
input_shape=(512,512,1)
loss = 'binary_crossentropy'
batch_size = 7
steps_per_epoch = training_dataset_size//batch_size
epochs = 100
start_epoch = 0
last_step = 10
clear_all = False

## Setup optimizer

In [9]:
optimizer = keras.optimizers.Adam(lr = learning_rate)

In [10]:
builder = Unet(input_shape=input_shape)
model = builder.build_model()
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
CNT_1_conv_1 (Conv2D)           (None, 512, 512, 64) 640         input[0][0]                      
__________________________________________________________________________________________________
CNT_1_conv_2 (Conv2D)           (None, 512, 512, 64) 36928       CNT_1_conv_1[0][0]               
__________________________________________________________________________________________________
CNT_2_maxpool (MaxPooling2D)    (None, 256, 256, 64) 0           CNT_1_conv_2[0][0]               
___________________________________________________________________________________________

## Dataset generators

In [11]:
train_generator = dataset.generator('training', batch_size=batch_size)
valid_generator = dataset.generator('validation', batch_size=1)

## Callbacks

In [12]:
tensor_board_directory = f'{tensor_board_root_directory}/{loss}-{learning_rate}'

In [13]:
# tensor board
tensorboard = keras.callbacks.TensorBoard(log_dir=tensor_board_directory, histogram_freq=0, write_graph=True, write_images=True, update_freq=100)
# image prediction history to be shown in the Tensor board
v_generator = dataset.generator('validation', batch_size=1)
v_data = [next(v_generator) for item in range(3)]
image_history = ImageHistory(tensor_board_dir=tensor_board_directory, data=v_data, last_step=last_step, draw_interval=100)
# model saving at end of an epoch
model_checkpoint = keras.callbacks.ModelCheckpoint(weight_file, monitor='loss',verbose=1, save_best_only=True)

In [14]:
def pixel_difference(y_true, y_pred):
    '''
    Custom metrics for comparison of images
    pixel by pixel. 
    '''
    cof = 100/(input_shape[0]*input_shape[1]*batch_size)
    return cof*K.sum(K.abs(y_true - y_pred))

metrics = ['accuracy', pixel_difference]

In [15]:
model.compile(optimizer=optimizer, loss = loss, metrics = metrics)

## Train Model

In [ ]:
model.fit_generator(generator=train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, initial_epoch=start_epoch,
                    validation_data=v_generator, validation_steps=validation_dataset_size,
                    callbacks=[model_checkpoint, tensorboard, image_history])




Epoch 1/100
 15/349 [>.............................] - ETA: 20:53 - loss: 0.5628 - accuracy: 0.9817 - pixel_difference: 37.5629WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/dl/lib/python3.7/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.

349/349 [==============================] - 777s 2s/step - loss: 0.0935 - accuracy: 0.9830 - pixel_difference: 4.9953 - val_loss: 0.1163 - val_accuracy: 0.9838 - val_pixel_difference: 0.3833

Epoch 00001: loss improved from inf to 0.09350, saving model to ./weights.hdf5
Epoch 2/100
349/349 [==============================] - 751s 2s/step - loss: 0.0627 - accuracy: 0.9830 - pixel_difference: 3.1255 - val_loss: 0.0955 - val_accuracy: 0.9838 - val_pixel_difference: 0.2642

Epoch 00002: loss improved from 0.09350 to 0.06269, saving model to ./weights.hdf5
Epoch 3/100
349/349 [==============================] - 750s 2s/step - loss: 0.0579 - accuracy: 0.9830 - pixel_dif